In [1]:
import os
import json
import nltk
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from pyserini.search.lucene import LuceneSearcher, querybuilder

d:\assignment-6\venvilham\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#download nltk
nltk.download('punkt_tab')
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))

#PREPROCESSING FUNCTION
def preprocess_text(text):
    text = text.lower()
    tokens = word_tokenize(text)
    processed_tokens = [word for word in tokens if word.isalpha() and word not in stop_words]
    return " ".join(processed_tokens)

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\ilham\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ilham\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
documents = [
    {"id": "d1", "contents": "The cat chased a small mouse into the garden."},
    {"id": "d2", "contents": "A friendly dog played fetch by the river."},
    {"id": "d3", "contents": "BM25 is a ranking function widely used in search engines."},
    {"id": "d4", "contents": "Boolean retrieval uses logical operators like AND and OR."},
    {"id": "d5", "contents": "TF-IDF weights terms by frequency and rarity."},
    {"id": "d6", "contents": "Neural retrieval uses dense embeddings for semantic search."},
    {"id": "d7", "contents": "The dog and the cat slept on the same couch."},
    {"id": "d8", "contents": "The library hosts a workshop on information retrieval."},
    {"id": "d9", "contents": "Students implemented BM25 and compared it with TF-IDF."},
    {"id": "d10", "contents": "The chef roasted chicken with rosemary and garlic."},
    {"id": "d11", "contents": "A black cat crossed the old stone bridge at night."},
    {"id": "d12", "contents": "Dogs are loyal companions during long hikes."},
    {"id": "d13", "contents": "The dataset contains fifteen short sentences for testing."},
    {"id": "d14", "contents": "Reranking models reorder BM25 candidates using transformers."},
    {"id": "d15", "contents": "The dog sniffed a cat but ignored the mouse."}
]

In [4]:
preprocessed_documents = []
for doc in documents:
    cleaned_contents = preprocess_text(doc['contents'])
    preprocessed_documents.append({"id": doc['id'], "contents": cleaned_contents})

# Simpan ke JSONL agar bisa diindeks Pyserini
os.makedirs('dataset_jsonlgg', exist_ok=True)
file_path = 'dataset_jsonlgg/documents_preprocessed.jsonl'

with open(file_path, 'w') as f:
    for doc in preprocessed_documents:
        f.write(json.dumps(doc) + '\n')

print(f"Dataset berhasil diproses dan disimpan di '{file_path}'\n")

Dataset berhasil diproses dan disimpan di 'dataset_jsonlgg/documents_preprocessed.jsonl'



In [ ]:
import math
from collections import defaultdict

doc_texts = [doc['contents'] for doc in preprocessed_documents]
doc_ids = [doc['id'] for doc in preprocessed_documents]
N = len(doc_texts) # Total number of documents

# 1. Hitung Document Frequency (DF)
df = defaultdict(int)
for text in doc_texts:
    unique_terms = set(text.split())
    for term in unique_terms:
        df[term] += 1

# 2. Hitung Inverse Document Frequency (IDF)
idf = {}
for term, freq in df.items():
    if freq > 0:
        idf[term] = math.log10(N / freq)

# 3. Hitung Term Frequency (TF) dan Vektor Dokumen (TF-IDF)
doc_vectors = []
for text in doc_texts:
    term_counts = defaultdict(int)
    for term in text.split():
        term_counts[term] += 1

    vector = {}
    for term, count in term_counts.items():
        tf_weight = 1 + math.log10(count) if count > 0 else 0
        vector[term] = tf_weight * idf.get(term, 0) 

    doc_vectors.append(vector)

# 4. Fungsi Perhitungan Cosine Similarity Manual
def cosine_similarity_search(query, doc_vectors, doc_ids, idf, N):
    processed_query = preprocess_text(query)
    query_terms = processed_query.split()

    if not query_terms:
        return []

    # Hitung Query Vector 
    query_tf_counts = defaultdict(int)
    for term in query_terms:
        query_tf_counts[term] += 1

    query_vector = {}
    for term, count in query_tf_counts.items():
        tf_weight = 1 + math.log10(count) if count > 0 else 0
        query_vector[term] = tf_weight * idf.get(term, 0)

    # Hitung Magnitudo Query
    q_magnitude = math.sqrt(sum([w**2 for w in query_vector.values()]))
    similarities = []
    doc_magnitudes = [math.sqrt(sum([w**2 for w in doc_vec.values()])) for doc_vec in doc_vectors]

    for i, (doc_vec, d_magnitude) in enumerate(zip(doc_vectors, doc_magnitudes)):
        dot_product = 0
        
        for term, q_weight in query_vector.items():
            if term in doc_vec:
                d_weight = doc_vec[term]
                dot_product += q_weight * d_weight

        # Hitung Similarity
        if dot_product > 0 and q_magnitude > 0 and d_magnitude > 0:
            sim = dot_product / (q_magnitude * d_magnitude)
            similarities.append((doc_ids[i], sim))
        elif dot_product > 0 and (q_magnitude == 0 or d_magnitude == 0):
            continue
        else:
            continue
    
    ranked = sorted(similarities, key=lambda x: x[1], reverse=True)
    return [item for item in ranked if item[1] > 0]

In [6]:
import sys
import subprocess
cmd = [
    sys.executable, "-m", "pyserini.index",
    "--collection", "JsonCollection",
    "--input", "dataset_jsonlgg",
    "--index", "my_index",
    "--generator", "DefaultLuceneDocumentGenerator",
    "--threads", "1",
    "--storePositions",
    "--storeDocvectors",
    "--storeRaw"
]

result = subprocess.run(cmd, capture_output=True, text=True)
print(result.stdout)
print(result.stderr)

pyserini.index is deprecated, please use pyserini.index.lucene.
2025-10-28 20:38:04,337 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:205) - Setting log level to INFO
2025-10-28 20:38:04,340 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:208) - ============ Loading Index Configuration ============
2025-10-28 20:38:04,340 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:209) - AbstractIndexer settings:
2025-10-28 20:38:04,340 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:210) -  + DocumentCollection path: dataset_jsonlgg
2025-10-28 20:38:04,340 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:211) -  + CollectionClass: JsonCollection
2025-10-28 20:38:04,340 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:212) -  + Index path: my_index
2025-10-28 20:38:04,340 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:213) -  + Threads: 1
2025-10-28 20:38:04,340 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:214) -  + Optimi

In [7]:
# 7. LOAD SEARCHER UNTUK BOOLEAN QUERY
from pyserini.search.lucene import querybuilder, LuceneSearcher

try:
    searcher = LuceneSearcher('my_index')
except Exception as e:
    print(f"Error: {e}")
    raise e

# Operator Boolean
MUST = querybuilder.JBooleanClauseOccur['must'].value
SHOULD = querybuilder.JBooleanClauseOccur['should'].value
MUST_NOT = querybuilder.JBooleanClauseOccur['must_not'].value

# Helper untuk membuat query Boolean
def make_query(terms, logic):
    builder = querybuilder.get_boolean_query_builder()
    for term, op in zip(terms, logic):
        builder.add(querybuilder.get_term_query(term), op)
    return builder.build()

# 8. DEFINISI QUERY
queries = {}

queries["dog AND cat"] = make_query(["dog", "cat"], [MUST, MUST])
queries["dog OR cat"] = make_query(["dog", "cat"], [SHOULD, SHOULD])
queries["dog AND NOT cat"] = make_query(["dog", "cat"], [MUST, MUST_NOT])

# (bm25 OR tf-idf) AND retrieval
inner4 = make_query(["bm25", "tf-idf"], [SHOULD, SHOULD])
outer4 = querybuilder.get_boolean_query_builder()
outer4.add(inner4, MUST)
outer4.add(querybuilder.get_term_query("retrieval"), MUST)
queries["(bm25 OR tf-idf) AND retrieval"] = outer4.build()

# model AND (retrieval OR bm25)
inner5 = make_query(["retrieval", "bm25"], [SHOULD, SHOULD])
outer5 = querybuilder.get_boolean_query_builder()
outer5.add(inner5, MUST)
outer5.add(querybuilder.get_term_query("model"), MUST)
queries["model AND (retrieval OR bm25)"] = outer5.build()

queries["mouse AND garden"] = make_query(["mouse", "garden"], [MUST, MUST])
queries["engine OR mouse"] = make_query(["engine", "mouse"], [SHOULD, SHOULD])
queries["sniff OR mouse"] = make_query(["sniff", "mouse"], [SHOULD, SHOULD])


In [8]:
for query_name, query_obj in queries.items():
    print("\n" + "="*80)
    print(f"QUERY: {query_name}")
    print("="*80)

    # BOOLEAN RETRIEVAL
    hits = searcher.search(query_obj)
    print("=== BOOLEAN RETRIEVAL RESULTS ===")
    if hits:
        for hit in hits:
            doc_data = next((d for d in documents if d["id"] == hit.docid), None)
            if doc_data:
                content = doc_data["contents"]
                docid_str = doc_data["id"]
            else:
                content = "N/A"
                docid_str = hit.docid
            print(f" {docid_str} | score={hit.score:.4f} | contents={content}")
    else:
        print(" (tidak ada hasil)")

    # COSINE SIMILARITY (TF-IDF)
    print("\n=== COSINE SIMILARITY (TF-IDF) RESULTS ===")
    # --- PERUBAHAN DI SINI ---
    cos_results = cosine_similarity_search(query_name, doc_vectors, doc_ids, idf, N) 
    # -------------------------
    if cos_results:
        # Batasi hasil Cosine Similarity hingga jumlah hits Boolean atau maksimal 5
        limit = len(hits) if hits else 5
        for docid, score in cos_results[:limit]: 
            doc_data = next((d for d in documents if d["id"] == docid), None)
            if doc_data:
                content = doc_data["contents"]
                docid_str = doc_data["id"]
            else:
                content = "N/A"
                docid_str = docid
            print(f" {docid_str} | score={score:.4f} | contents={content}")
    else:
        print(" (tidak ada hasil)")

print("\nSelesai menampilkan hasil untuk semua query.")


QUERY: dog AND cat
=== BOOLEAN RETRIEVAL RESULTS ===
 d7 | score=1.3990 | contents=The dog and the cat slept on the same couch.
 d15 | score=1.3482 | contents=The dog sniffed a cat but ignored the mouse.

=== COSINE SIMILARITY (TF-IDF) RESULTS ===
 d7 | score=0.4777 | contents=The dog and the cat slept on the same couch.
 d15 | score=0.4337 | contents=The dog sniffed a cat but ignored the mouse.

QUERY: dog OR cat
=== BOOLEAN RETRIEVAL RESULTS ===
 d7 | score=1.3990 | contents=The dog and the cat slept on the same couch.
 d15 | score=1.3482 | contents=The dog sniffed a cat but ignored the mouse.
 d1 | score=0.6741 | contents=The cat chased a small mouse into the garden.
 d12 | score=0.6741 | contents=Dogs are loyal companions during long hikes.
 d2 | score=0.6741 | contents=A friendly dog played fetch by the river.
 d11 | score=0.6284 | contents=A black cat crossed the old stone bridge at night.

=== COSINE SIMILARITY (TF-IDF) RESULTS ===
 d7 | score=0.4777 | contents=The dog and the 